In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
housing=spark.read.option("header",True).option("inferschema",True).csv("/mnt/housing/data/raw/housing.csv")

In [0]:
housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [0]:
from pyspark.sql.functions import col,isnan, when, count
housing.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in housing.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [0]:
housing=housing.na.drop("all", subset=["total_bedrooms"])

In [0]:
from pyspark.sql.functions import col,isnan, when, count
housing.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in housing.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|             0|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [0]:
housing=housing.withColumn("rooms_per_household",col("total_rooms")/col("households"))
housing=housing.withColumn("bedrooms_per_household",col("total_bedrooms")/col("households"))
housing=housing.withColumn("population_per_household",col("population")/col("households"))

In [0]:
housing.count()

Out[38]: 20433

In [0]:
housing.write.option("header", "true").option("schema","true").mode('overwrite').csv('/mnt/housing/data/cleaned/housing_cleaned/')

In [0]:
datalocation='/mnt/housing/data/cleaned/housing_cleaned/'
files=dbutils.fs.ls(datalocation)
csv_file=[x.path for x in files if x.path.endswith(".csv")][0]
dbutils.fs.mv(csv_file,datalocation.rstrip('/')+".csv")
dbutils.fs.rm(datalocation,recurse=True)

Out[45]: True